## Simulating melts of polymer chains

### Pre-sim setup

In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import gsd.hoomd
from hoomd.md import nlist

import unyt as u
import glob
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
import os

import flowermd
from flowermd.utils import get_target_box_mass_density
from flowermd.base import Simulation, Molecule
from flowermd.library import FF_from_file
from flowermd.base.system import Pack

from polymer_dictionary import polymer_dictionary

# Rerun with the original seed first

In [2]:
# Updates atom names to fit with hoomd conventions
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

In [3]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = os.getcwd() # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [3]:
mol2_path = os.getcwd() + "/mol2/"
xml_path = os.getcwd() + "/xml/"

## Single molecule sim:

In [14]:
system_file = mol2_path + "P3HT.mol2"
ff_filepath = xml_path + "P3HT.xml"

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)

No charged group detected, skipping electrostatics.


In [15]:
system.hoomd_snapshot
cpu = hoomd.device.CPU()
sim = Simulation.from_system(system=system, gsd_write_freq=5000, log_write_freq=5000, device=cpu, gsd_file_name="test.gsd",log_file_name="test.txt")

Initializing simulation state from a gsd.hoomd.Frame.


In [16]:
sim.run_NVT(n_steps=5e4, kT=5.0, tau_kt=0.05)
sim.flush_writers()

Step 5000 of 50000; TPS: 19076.76; ETA: 0.0 minutes
Step 10000 of 50000; TPS: 33860.77; ETA: 0.0 minutes
Step 15000 of 50000; TPS: 45340.51; ETA: 0.0 minutes
Step 20000 of 50000; TPS: 55306.52; ETA: 0.0 minutes
Step 25000 of 50000; TPS: 63483.03; ETA: 0.0 minutes
Step 30000 of 50000; TPS: 70213.38; ETA: 0.0 minutes
Step 35000 of 50000; TPS: 75901.99; ETA: 0.0 minutes
Step 40000 of 50000; TPS: 81356.04; ETA: 0.0 minutes
Step 45000 of 50000; TPS: 86286.41; ETA: 0.0 minutes


In [24]:
sim.reference_values

{'energy': unyt_quantity(1.046, 'kJ/mol'),
 'length': unyt_quantity(0.35635949, 'nm'),
 'mass': unyt_quantity(32.06, 'amu')}

In [40]:
energy = ((1046/6.022*10**23)*J).to('J')
length = (0.35635949*nm).to('m')
mass = (32.06*amu).to('kg')

In [41]:
tau = np.sqrt((mass*length**2)/energy)

In [45]:
print(tau)

1.9728730976366816e-35 sqrt(kg)*m/sqrt(J)


## Running sims of every polymer in the Danielsen et al. paper

In [20]:
pack_seeds = np.linspace(1800, 2000, 3, dtype=int)
pack_seeds = pack_seeds.tolist()
print(pack_seeds)

[1800, 1900, 2000]


In [ ]:
for pack_seed in pack_seeds:
    for i in range(len(key_list)):
        system_file = mol2_path + "10_mers/" + key_list[i] + "_10mer.mol2"
        ff_filepath = xml_path + key_list[i] + ".xml"
        gsd_path = os.getcwd() + "/gsd_files/10_mers/"+str(pack_seed)+"/"
        num_mers = "_10mer" # The number of monomers being simulated is determined by the mol2, this is for naming the resulting gsd and .txt files
        espmol = espaloma_mol(system_file)
        molecule = Molecule(num_mols=1, compound=espmol)
        cpu = hoomd.device.CPU()
        
        molff = FF_from_file(ff_filepath)
        system = Pack(molecules=molecule,density=0.01 * u.g/u.cm**3,seed=pack_seed, packing_expand_factor=5)
        system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
        system.hoomd_snapshot
        hoomd_forces = system.hoomd_forcefield
        hoomd_forces
        lj_force = hoomd_forces[3]
        sim = Simulation.from_system(system=system, gsd_write_freq=5000, log_write_freq=5000, device=cpu, gsd_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".gsd",log_file_name=gsd_path+key_list[i]+num_mers+"_seed_"+str(pack_seed)+".txt")
        sim.run_NVT(n_steps=5e5, kT=5.0, tau_kt=1.0)
        sim.flush_writers()

No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 5000 of 500000; TPS: 556.02; ETA: 14.8 minutes
Step 10000 of 500000; TPS: 667.54; ETA: 12.2 minutes
Step 15000 of 500000; TPS: 714.2; ETA: 11.3 minutes
Step 20000 of 500000; TPS: 728.63; ETA: 11.0 minutes
Step 25000 of 500000; TPS: 752.93; ETA: 10.5 minutes
Step 30000 of 500000; TPS: 765.83; ETA: 10.2 minutes
Step 35000 of 500000; TPS: 737.95; ETA: 10.5 minutes
Step 40000 of 500000; TPS: 746.45; ETA: 10.3 minutes
